![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_E5.ipynb)

# Import OpenVINO E5 models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting E5 models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support for this annotator was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for E5 from HuggingFace and they have to be in `Sentence Similarity` category. Meaning, you cannot use E5 models trained/fine-tuned on a specific task such as token/sequence classification.

## 1. Export and Save HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.

- We lock `transformers` on version `4.48.3`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [ ]:
!pip install -q --upgrade transformers==4.48.3 optimum[openvino]

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [intfloat/e5-small-v2](https://huggingface.co/intfloat/e5-small-v2) model from HuggingFace as an example and load it as a `OVModelForFeatureExtraction`, representing an OpenVINO model.


In [2]:
from optimum.intel import OVModelForFeatureExtraction
from transformers import AutoTokenizer

MODEL_NAME = "intfloat/e5-small-v2"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

ov_model = OVModelForFeatureExtraction.from_pretrained(MODEL_NAME, export=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

ov_model.save_pretrained(EXPORT_PATH)
tokenizer.save_pretrained(EXPORT_PATH)

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse th

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


('ov_models/intfloat/e5-small-v2/tokenizer_config.json',
 'ov_models/intfloat/e5-small-v2/special_tokens_map.json',
 'ov_models/intfloat/e5-small-v2/vocab.txt',
 'ov_models/intfloat/e5-small-v2/added_tokens.json',
 'ov_models/intfloat/e5-small-v2/tokenizer.json')

('ov_models/intfloat/e5-small-v2/tokenizer_config.json',
 'ov_models/intfloat/e5-small-v2/special_tokens_map.json',
 'ov_models/intfloat/e5-small-v2/vocab.txt',
 'ov_models/intfloat/e5-small-v2/added_tokens.json',
 'ov_models/intfloat/e5-small-v2/tokenizer.json')

Let's have a look inside these two directories and see what we are dealing with:

In [3]:
!ls -l {EXPORT_PATH}

total 131036
-rw-r--r-- 1 root root       626 Jun 21 22:39 config.json
-rw-r--r-- 1 root root 132852896 Jun 21 22:39 openvino_model.bin
-rw-r--r-- 1 root root    363256 Jun 21 22:39 openvino_model.xml
-rw-r--r-- 1 root root       695 Jun 21 22:39 special_tokens_map.json
-rw-r--r-- 1 root root      1220 Jun 21 22:39 tokenizer_config.json
-rw-r--r-- 1 root root    711396 Jun 21 22:39 tokenizer.json
-rw-r--r-- 1 root root    231508 Jun 21 22:39 vocab.txt


We need to move `vocabs.txt` from the tokenizer to assets folder which Spark NLP will look for

In [4]:
!mkdir {EXPORT_PATH}/assets && mv {EXPORT_PATH}/vocab.txt {EXPORT_PATH}/assets/

In [5]:
!ls -l {EXPORT_PATH}/assets

total 228
-rw-r--r-- 1 root root 231508 Jun 21 22:39 vocab.txt


## 2. Import and Save E5 in Spark NLP

- Install and set up Spark NLP in Google Colab
- This example uses specific versions of `pyspark` and `spark-nlp` that have been tested with the transformer model to ensure everything runs smoothly.

In [6]:
!pip install -q pyspark==3.5.4 spark-nlp==5.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.7/635.7 kB 36.1 MB/s eta 0:00:00


Let's start Spark with Spark NLP included via our simple `start()` function

In [7]:
import sparknlp

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  5.5.3
Apache Spark version:  3.5.4


- Let's use `loadSavedModel` functon in `E5Embeddings` which allows us to load the OpenVINO model.
- Most params will be set automatically. They can also be set later after loading the model in `E5Embeddings` during runtime, so don't worry about setting them now.
- `loadSavedModel` accepts two params, first is the path to the exported model. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.st and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [8]:
from sparknlp.annotator import E5Embeddings

E5 = E5Embeddings.loadSavedModel(f"{EXPORT_PATH}", spark)\
    .setInputCols(["document"])\
    .setOutputCol("E5")

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [9]:
E5.write().overwrite().save(f"{MODEL_NAME}_spark_nlp")

Let's clean up stuff we don't need anymore

In [10]:
!rm -rf {EXPORT_PATH}

Awesome  😎 !

This is your OpenVINO E5 model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [11]:
! ls -l {MODEL_NAME}_spark_nlp

total 130124
-rw-r--r-- 1 root root 133236765 Jun 21 22:49 e5_openvino
drwxr-xr-x 3 root root      4096 Jun 21 22:49 fields
drwxr-xr-x 2 root root      4096 Jun 21 22:49 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny E5 model 😊

In [12]:
from sparknlp.base import DocumentAssembler
from sparknlp.annotator import E5Embeddings
from pyspark.ml import Pipeline

document_assembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

E5_loaded = E5Embeddings.load(f"{MODEL_NAME}_spark_nlp")\
    .setInputCols(["document"])\
    .setOutputCol("E5")

pipeline = Pipeline(stages=[
    document_assembler,
    E5_loaded
])

data = spark.createDataFrame([[
    'William Henry Gates III (born October 28, 1955) is an American business magnate, software developer, investor, and philanthropist.'
]]).toDF("text")

model = pipeline.fit(data)
result = model.transform(data)

result.selectExpr("explode(E5.embeddings) as embeddings").show()

+--------------------+
|          embeddings|
+--------------------+
|[-0.042928334, 0....|
+--------------------+



That's it! You can now go wild and use hundreds of E5 models from HuggingFace 🤗 in Spark NLP 🚀
